In [13]:
papeis = ['ITSA4','SAPR11', 'CAML3',  'EGIE3', 'CXSE3', 'BBSE3', 'FLRY3', 'TAEE11', 'TUPY3', 'ABCB4' , 'BBAS3' , 'PSSSA3' , 'AMBP3']
papeis.sort()

In [14]:
import pandas as pd
url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/main/data/SI_Acoes.csv'
tabela = pd.read_csv(url,sep=';' , decimal=',' , thousands='.')
tabela.head(2)

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,AALR3,15.89,NaN,-8.61,1.68,0.7,29.80,3.29,-18.37,48.11,...,0.42,0.58,0.44,1.84,NaN,708789.0,9.47,-1.85,0.36,1.864295e+09
1,ABCB4,23.52,6.49,6.52,0.98,0.1,16.03,9.38,10.00,6.95,...,0.10,0.90,0.15,20.08,16.19,12220779.0,24.07,3.61,-0.22,5.710291e+09


In [15]:
## usar uma porcentagem do cagr 5y dos lucros como proxy de retorno futur0
projetado = tabela.loc[tabela['TICKER'].isin(papeis)][['TICKER' , 'CAGR LUCROS 5 ANOS']].set_index('TICKER').fillna(0)/111
projetado.rename(columns={'CAGR LUCROS 5 ANOS' : 'ret_projetado'},inplace=True)
projetado

,ret_projetado
TICKER,
ABCB4,0.145856
AMBP3,0.000000
BBAS3,0.183604
BBSE3,0.169730
CAML3,0.074505
CXSE3,0.163874
EGIE3,0.122162
FLRY3,0.112432
ITSA4,0.078288


In [16]:
periodo = '5y'
intervalo = '1d'
ponto = 'Close'

#@ markdown Marque  a caixa se deseja usar retorno histórico,
#@ markdown caso contrário, usará o projetado
# Retorno_historico = False #@param ￼{type:"boolean"}

# parametros  para anualização
n_dias = 252

import numpy as np
import yfinance as yf
ativos = projetado.index.to_list()
n_ativos = len(ativos)

retorno_anual = projetado.copy()

In [17]:
# Download preços do yahoo finance
dados = yf.download( [tck+'.SA' for tck in ativos],
                    interval=intervalo,
                    auto_adjust=True,
                    progress=False )[ponto]

dados.columns = dados.columns.str.rstrip('.SA')
dados.index = dados.index.date
dados

Ticker,ABCB4,AMBP3,BBAS3,BBSE3,CAML3,CXSE3,EGIE3,FLRY3,ITSA4,SAPR11,TAEE11,TUPY3
2000-01-03,NaN,NaN,0.485438,NaN,NaN,NaN,NaN,NaN,0.397766,NaN,NaN,1272.323364
2000-01-04,NaN,NaN,0.462351,NaN,NaN,NaN,NaN,NaN,0.359788,NaN,NaN,1272.323364
2000-01-05,NaN,NaN,0.467087,NaN,NaN,NaN,NaN,NaN,0.389771,NaN,NaN,1272.323364
2000-01-06,NaN,NaN,0.473599,NaN,NaN,NaN,NaN,NaN,0.395767,NaN,NaN,1272.323364
2000-01-07,NaN,NaN,0.462943,NaN,NaN,NaN,NaN,NaN,0.393768,NaN,NaN,1272.323364
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-02,23.500000,66.300003,28.420000,36.709999,9.387545,16.230000,44.400002,15.770000,10.870000,28.690001,35.430000,27.650000
2024-09-03,23.400000,63.000000,28.680000,36.880001,9.496702,16.379999,44.660000,16.129999,11.040000,29.350000,35.709999,27.910000
2024-09-04,23.440001,65.940002,28.990000,37.419998,9.700000,15.940000,45.189999,16.709999,11.130000,29.120001,36.270000,28.020000
2024-09-05,23.639999,63.990002,29.250000,36.950001,9.790000,15.850000,44.880001,16.799999,11.230000,29.809999,35.840000,27.600000


In [18]:
# retorno histórico, tb usado coo proxy de retorno futuro
retornos = (dados.pct_change()).dropna()
retorno_anual['ret_historico']= retornos.mean()*n_dias

In [19]:
retorno_anual

,ret_projetado,ret_historico
TICKER,,
ABCB4,0.145856,0.254934
AMBP3,0.000000,0.505112
BBAS3,0.183604,0.319362
BBSE3,0.169730,0.256101
CAML3,0.074505,0.101032
CXSE3,0.163874,0.258045
EGIE3,0.122162,0.126958
FLRY3,0.112432,-0.020590
ITSA4,0.078288,0.179085


In [20]:
#Volatilidade = dp retorno diário
riscos = retornos.std().dropna()
# volatilidade anualizada
vol_ano = riscos*np.sqrt(n_dias)
#covariancia anualizada
matriz_covarianca_ano = retornos.cov()*n_dias

In [21]:
matriz_covarianca_ano

Ticker,ABCB4,AMBP3,BBAS3,BBSE3,CAML3,CXSE3,EGIE3,FLRY3,ITSA4,SAPR11,TAEE11,TUPY3
Ticker,,,,,,,,,,,,
ABCB4,0.092521,0.041211,0.037939,0.014958,0.035798,0.028427,0.015341,0.026803,0.033409,0.019379,0.014345,0.033274
AMBP3,0.041211,0.446114,0.029165,0.017645,0.043165,0.034178,0.020315,0.062974,0.033316,0.037476,0.022768,0.047573
BBAS3,0.037939,0.029165,0.071154,0.020870,0.023523,0.026424,0.015523,0.026130,0.038193,0.021038,0.014975,0.025560
BBSE3,0.014958,0.017645,0.020870,0.047172,0.016123,0.025972,0.013647,0.020069,0.017355,0.011585,0.012567,0.018713
CAML3,0.035798,0.043165,0.023523,0.016123,0.156122,0.019777,0.019134,0.028891,0.023963,0.024846,0.015638,0.030211
CXSE3,0.028427,0.034178,0.026424,0.025972,0.019777,0.087381,0.014404,0.024173,0.022212,0.018214,0.012275,0.028075
EGIE3,0.015341,0.020315,0.015523,0.013647,0.019134,0.014404,0.032648,0.018197,0.016561,0.017579,0.016489,0.014609
FLRY3,0.026803,0.062974,0.026130,0.020069,0.028891,0.024173,0.018197,0.097508,0.025819,0.023582,0.016116,0.028355
ITSA4,0.033409,0.033316,0.038193,0.017355,0.023963,0.022212,0.016561,0.025819,0.048628,0.017938,0.013516,0.024491


### ***otimização***

In [23]:
# caclula a volatilidade (aa) da carteira dada pelos pesos p
def volatilidade( p ):
  '''
  volatilidade da carteira definida por pesos p
  '''
  return  np.sqrt(np.dot(p.T,np.dot(matriz_covarianca_ano,p)))

# calcula a retorno esperado da carteira dada pelos pesos p
def retorno_esperado( p , tipo='p'):
  '''
  retorno da carteira definida por pesos p
  '''
  if tipo == 'p' : retornos = retorno_anual['ret_projetado']
  else : retornos = retorno_anual['ret_historico']

  return np.dot(retornos , p )

# a biblioteca de otimização tem um metodo de *minimização*
# para maximizar a função f calculamos o minimo de -f
def neg_ratio ( p , tipo='p') :
  sharpe = retorno_esperado(p,tipo) / volatilidade(p)
  return -sharpe


#### **carteira com a melhor razão retorno/risco**
**considerando que $0.05 \leq \text{peso} \leq 0.2$ e que as açoes de crescimento formam 30% da carteira**


In [48]:
limites   = tuple((0.05,.2) for x in range(n_ativos))     ## 0.05 ≤ peso ≤ 0.2

restricao = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},                ## a soma dos pesos é 1, i.e., np.sum(x) - 1 = 0
    {'type': 'ineq', 'fun' : lambda x: 0.3 -  x[1] - x[4] - x[7] - x[11]  } ## caml+flry+...<= 30% 
)

In [49]:
import scipy.optimize as scpopt

max_ratio = scpopt.minimize(neg_ratio,        # para max do ratio, calculamos min do -ratio = neg_ratio
                            x0 =  n_ativos * [1. / n_ativos] ,
                            args = 'p' ,
                            method='SLSQP',
                            bounds =  limites,
                            constraints =restricao
                            )

max_ratio_pesos_p = max_ratio['x']
print(max_ratio['message'])

Optimization terminated successfully


In [50]:
import scipy.optimize as scpopt

max_ratio = scpopt.minimize(neg_ratio,        # para max do ratio, calculamos min do -ratio = neg_ratio
                            x0 =  n_ativos * [1. / n_ativos] ,
                            args = 'h' ,
                            method='SLSQP',
                            bounds =  limites,
                            constraints =restricao
                            )

max_ratio_pesos_h = max_ratio['x']
print(max_ratio['message'])

Optimization terminated successfully


projetado

In [51]:
max_ratio_r = np.round( retorno_esperado(max_ratio_pesos_p,'p') ,4)
max_ratio_v = np.round( volatilidade(max_ratio_pesos_p),4)
max_ratio_s = np.round( -max_ratio['fun'],4)

# resultado
for x, y in zip(ativos, max_ratio_pesos_p):
  print(f'{x}: {100*y:.2f}%')
print('')
print('Retorno esperado %.2f' %(max_ratio_r*100)+'%')
print('Volatilidade %.2f' %(max_ratio_v*100)+'%')

ABCB4: 5.00%
AMBP3: 5.00%
BBAS3: 16.47%
BBSE3: 20.00%
CAML3: 5.00%
CXSE3: 6.58%
EGIE3: 16.95%
FLRY3: 5.00%
ITSA4: 5.00%
SAPR11: 5.00%
TAEE11: 5.00%
TUPY3: 5.00%

Retorno esperado 12.92%
Volatilidade 16.14%


historico

In [52]:
max_ratio_r = np.round( retorno_esperado(max_ratio_pesos_h,'h') ,4)
max_ratio_v = np.round( volatilidade(max_ratio_pesos_h),4)
max_ratio_s = np.round( -max_ratio['fun'],4)

# resultado
for x, y in zip(ativos, max_ratio_pesos_h):
  print(f'{x}: {100*y:.2f}%')
print('')
print('Retorno esperado %.2f' %(max_ratio_r*100)+'%')
print('Volatilidade %.2f' %(max_ratio_v*100)+'%')

ABCB4: 5.00%
AMBP3: 7.18%
BBAS3: 20.00%
BBSE3: 20.00%
CAML3: 5.00%
CXSE3: 9.30%
EGIE3: 5.00%
FLRY3: 5.00%
ITSA4: 5.00%
SAPR11: 8.52%
TAEE11: 5.00%
TUPY3: 5.00%

Retorno esperado 23.46%
Volatilidade 17.22%


**com qualquer proporção de ativos**

In [53]:
max = scpopt.minimize(neg_ratio, x0 =  n_ativos * [1. / n_ativos] ,args='p',  method='SLSQP',
                            bounds =  tuple((0,1) for asset in range(n_ativos)),
                            constraints =( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})    )
max_pesos_p = max['x']

max_r = round( retorno_esperado(max_pesos_p, 'p'),4)
max_v = round( volatilidade(max_pesos_p),4)
max_s = round( -max['fun'],4)

# resultado
for x, y in zip(ativos, max_pesos_p):
  print(f'{x}: {100*y:.2f}%')
print(' ')
print('Retorno esperado %.2f' %(max_r*100)+'%')
print('Volatilidade %.2f' %(max_v*100)+'%')

ABCB4: 4.43%
AMBP3: 0.00%
BBAS3: 19.85%
BBSE3: 35.76%
CAML3: 0.00%
CXSE3: 7.54%
EGIE3: 31.57%
FLRY3: 0.00%
ITSA4: 0.00%
SAPR11: 0.85%
TAEE11: 0.00%
TUPY3: 0.00%
 
Retorno esperado 15.54%
Volatilidade 16.02%


In [54]:
max = scpopt.minimize(neg_ratio, x0 =  n_ativos * [1. / n_ativos] ,args='h',  method='SLSQP',
                            bounds =  tuple((0,1) for asset in range(n_ativos)),
                            constraints =( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})    )
max_pesos_h = max['x']

max_r = round( retorno_esperado(max_pesos_h,'h'),4)
max_v = round( volatilidade(max_pesos_h),4)
max_s = round( -max['fun'],4)

# resultado
for x, y in zip(ativos, max_pesos_h):
  print(f'{x}: {100*y:.2f}%')
print(' ')
print('Retorno esperado %.2f' %(max_r*100)+'%')
print('Volatilidade %.2f' %(max_v*100)+'%')

ABCB4: 6.10%
AMBP3: 7.66%
BBAS3: 28.16%
BBSE3: 39.15%
CAML3: 0.00%
CXSE3: 6.59%
EGIE3: 0.00%
FLRY3: 0.00%
ITSA4: 0.00%
SAPR11: 12.35%
TAEE11: 0.00%
TUPY3: 0.00%
 
Retorno esperado 28.66%
Volatilidade 18.24%


#### **Carteira de menor volatilidade**


**considerando que $0.05 \leq \text{peso} \leq 0.2$ e que as açoes de crescimento formam 30% da carteira**

In [55]:
min_vol_P = scpopt.minimize(volatilidade,
                          x0 =  n_ativos * [1. / n_ativos] ,
                          method='SLSQP',
                          bounds = limites ,
                          constraints = restricao)

min_vol_pesos_P = min_vol_P['x']

min_volP_r = round( retorno_esperado(min_vol_pesos_P,'p'),4)
min_volP_v = round( min_vol_P['fun'],4)
min_volP_s = round( -neg_ratio(min_vol_pesos_P),4)

for x, y in zip(ativos, min_vol_pesos_P):
  print(f'{x}: {100*y:.2f}% ')
print(' ')
print('Retorno esperado %.2f' %(100*min_volP_r)+'%')
print('Volatilidade %.2f' %(100*min_volP_v)+'%')

ABCB4: 5.00% 
AMBP3: 5.00% 
BBAS3: 5.00% 
BBSE3: 14.98% 
CAML3: 5.00% 
CXSE3: 5.00% 
EGIE3: 20.00% 
FLRY3: 5.00% 
ITSA4: 5.00% 
SAPR11: 5.02% 
TAEE11: 20.00% 
TUPY3: 5.00% 
 
Retorno esperado 11.12%
Volatilidade 15.15%


In [56]:
min_vol_P = scpopt.minimize(volatilidade,
                          x0 =  n_ativos * [1. / n_ativos] ,
                          method='SLSQP',
                          bounds = limites ,
                          constraints = restricao)

min_vol_pesos_P = min_vol_P['x']

min_volP_r = round( retorno_esperado(min_vol_pesos_P,'h'),4)
min_volP_v = round( min_vol_P['fun'],4)
min_volP_s = round( -neg_ratio(min_vol_pesos_P),4)

for x, y in zip(ativos, min_vol_pesos_P):
  print(f'{x}: {100*y:.2f}% ')
print(' ')
print('Retorno esperado %.2f' %(100*min_volP_r)+'%')
print('Volatilidade %.2f' %(100*min_volP_v)+'%')

ABCB4: 5.00% 
AMBP3: 5.00% 
BBAS3: 5.00% 
BBSE3: 14.98% 
CAML3: 5.00% 
CXSE3: 5.00% 
EGIE3: 20.00% 
FLRY3: 5.00% 
ITSA4: 5.00% 
SAPR11: 5.02% 
TAEE11: 20.00% 
TUPY3: 5.00% 
 
Retorno esperado 17.43%
Volatilidade 15.15%


**considerando qualquer proporção de ativos**


In [57]:
min_vol = scpopt.minimize(volatilidade,
                          x0 =  n_ativos * [1. / n_ativos] ,
                          method='SLSQP',
                          bounds = tuple((0,1) for asset in range(n_ativos)),
                          constraints = ( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})    )

min_vol_pesos = min_vol['x']

min_vol_r = round( retorno_esperado(min_vol_pesos,'p'),4)
min_vol_v = round( min_vol['fun'],4)
min_vol_s = round( -neg_ratio(min_vol_pesos,'p'),4)

for x, y in zip(ativos, min_vol_pesos):
  print(f'{x}: {100*y:.2f}% ')
print(' ')
print('Retorno esperado %.2f' %(min_vol_r*100)+'%')
print('Volatilidade %.2f' %(min_vol_v*100)+'%')

ABCB4: 2.14% 
AMBP3: 0.00% 
BBAS3: 0.00% 
BBSE3: 16.78% 
CAML3: 0.52% 
CXSE3: 3.25% 
EGIE3: 23.65% 
FLRY3: 0.11% 
ITSA4: 9.81% 
SAPR11: 7.42% 
TAEE11: 33.53% 
TUPY3: 2.80% 
 
Retorno esperado 10.73%
Volatilidade 14.06%


In [58]:
min_vol = scpopt.minimize(volatilidade,
                          x0 =  n_ativos * [1. / n_ativos] ,
                          method='SLSQP',
                          bounds = tuple((0,1) for asset in range(n_ativos)),
                          constraints = ( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})    )

min_vol_pesos = min_vol['x']

min_vol_r = round( retorno_esperado(min_vol_pesos,'h'),4)
min_vol_v = round( min_vol['fun'],4)
min_vol_s = round( -neg_ratio(min_vol_pesos,'h'),4)

for x, y in zip(ativos, min_vol_pesos):
  print(f'{x}: {100*y:.2f}% ')
print(' ')
print('Retorno esperado %.2f' %(min_vol_r*100)+'%')
print('Volatilidade %.2f' %(min_vol_v*100)+'%')

ABCB4: 2.14% 
AMBP3: 0.00% 
BBAS3: 0.00% 
BBSE3: 16.78% 
CAML3: 0.52% 
CXSE3: 3.25% 
EGIE3: 23.65% 
FLRY3: 0.11% 
ITSA4: 9.81% 
SAPR11: 7.42% 
TAEE11: 33.53% 
TUPY3: 2.80% 
 
Retorno esperado 14.75%
Volatilidade 14.06%
